In [5]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col, datediff, current_date, round, dense_rank, when
import os


spark = SparkSession.builder.master('local[4]').getOrCreate()


In [6]:
# Get input file sizes in MB
total = 0
for file in os.listdir('../corporate_uk'):
    print(file, os.path.getsize(f'../corporate_uk/{file}') / (1024**2))
    total += os.path.getsize(f'../corporate_uk/{file}') / (1024**2)
print(total)

companies.csv 1416.655816078186
companies_sic_codes.csv 1591.1925315856934
filings.csv 2041.9274320602417
officers_and_owners.csv 1697.7515335083008
6747.527313232422


In [7]:
companies = spark.read.option('delimiter', ';').option('header', 'true').csv('../corporate_uk/companies.csv')\
    .withColumn('current_assets', col('current_assets').cast('double'))\
    .withColumn('average_number_employees_during_period', col('average_number_employees_during_period').cast('double'))\
    .withColumn('account_type', when(col('account_type') == 'Null', None).otherwise(col('account_type')))
sic_codes = spark.read.option('delimiter', ';').option('header', 'true').csv('../corporate_uk/companies_sic_codes.csv')
filings = spark.read.option('delimiter', ';').option('header', 'true').csv('../corporate_uk/filings.csv')\
    .withColumn('pages', col('pages').cast('int'))
owners = spark.read.option('delimiter', ';').option('header', 'true').csv('../corporate_uk/officers_and_owners.csv')

In [8]:
companies.groupBy('company_status')\
    .count()\
    .sort('count', ascending=False)\
    .show(truncate=False)

+----------------------+-------+
|company_status        |count  |
+----------------------+-------+
|Active                |4744110|
|Dissolved             |534818 |
|Liquidation           |110597 |
|Registered            |29219  |
|In Administration     |3612   |
|Receiver Action       |2683   |
|Open                  |1075   |
|Removed               |1035   |
|Converted / Closed    |591    |
|Voluntary Arrangement |513    |
|Insolvency Proceedings|345    |
|Closed                |302    |
+----------------------+-------+



In [9]:
# Which account type is most common in each jurisdiction?
w = Window.partitionBy(['jurisdiction']).orderBy(col('count').desc())
companies.filter(col('jurisdiction').isNotNull() & col('account_type').isNotNull())\
        .groupBy(['jurisdiction', 'account_type'])\
        .count()\
        .withColumn('rnk', dense_rank().over(w))\
        .where(col('rnk') == 1)\
        .drop(col('rnk'))\
        .sort(col('jurisdiction'))\
        .show(truncate=False)

+----------------+------------+-------+
|jurisdiction    |account_type|count  |
+----------------+------------+-------+
|England/Wales   |Micro Entity|1711811|
|European Union  |Micro Entity|1      |
|Northern Ireland|Micro Entity|30758  |
|Scotland        |Micro Entity|88516  |
|United Kingdom  |Full        |5906   |
|Wales           |Micro Entity|12363  |
+----------------+------------+-------+



In [10]:
sic_codes.groupBy(['sic_section', 'sic_division'])\
    .count()\
    .sort(col('count').desc())\
    .limit(10)\
    .show(truncate=False)

+--------------------------------------------------------------------+---------------------------------------------------------------------------+------+
|sic_section                                                         |sic_division                                                               |count |
+--------------------------------------------------------------------+---------------------------------------------------------------------------+------+
|Real estate activities                                              |Real estate activities                                                     |831302|
|Wholesale and retail trade; repair of motor vehicles and motorcycles|Retail trade, except of motor vehicles and motorcycles                     |690760|
|Construction                                                        |Specialised construction activities                                        |413583|
|Information and communication                                       |Comput

In [11]:
companies.withColumn('age', datediff(current_date(), col('incorporation_date')) / 365.25)\
.select([
    'company_type', 
    'age'])\
.groupBy('company_type')\
.avg()\
.withColumn('average_age', round(col('avg(age)'), 2))\
.drop(col('avg(age)'))\
.sort(col('average_age'))\
.show(truncate=False)

+--------------------------------------------------------------------------------------+-----------+
|company_type                                                                          |average_age|
+--------------------------------------------------------------------------------------+-----------+
|Overseas entity                                                                       |1.83       |
|Further education or sixth form college corporation                                   |4.16       |
|Protected cell company                                                                |4.78       |
|Scottish qualifying partnership                                                       |5.54       |
|UK establishment company                                                              |7.14       |
|Limited liability partnership                                                         |9.26       |
|Private limited company                                                               |9.4

In [12]:
companies.filter((companies.next_accounts_overdue == True) | (companies.confirmation_statement_overdue == True))\
    .select(col('company_name'))\
    .distinct()\
    .show()

+--------------------+
|        company_name|
+--------------------+
|  Jensen Telecom Ltd|
| Barrow Boys Limited|
|Seagull Internati...|
|  Deborah Coates Ltd|
|2mc Holdings Limited|
|Sunny Sandcastles...|
|     Uneed2relax Ltd|
|Serhat Wholesale ...|
|         Demodem Ltd|
|Florin Contracts ...|
|Cullingford Carpe...|
|Woodstock Product...|
|Waves Hotels Grou...|
|  Shollwaves Limited|
|Industrialize Uk Ltd|
|Whole Recruitment...|
|  Bertea Ipswich Ltd|
|Focus Industry An...|
|Mns Retail Interi...|
|       Sdhac Limited|
+--------------------+
only showing top 20 rows



In [13]:
companies.groupBy('account_type')\
    .count()\
    .withColumn('percentage', round(100 * col('count') / companies.count(), 3))\
    .sort('percentage', ascending=False)\
    .show(truncate=False)

+---------------------------+-------+----------+
|account_type               |count  |percentage|
+---------------------------+-------+----------+
|Micro Entity               |1843452|33.956    |
|Total Exemption Full       |1346638|24.805    |
|NULL                       |1083040|19.95     |
|Dormant                    |742074 |13.669    |
|Unaudited abridged         |175651 |3.235     |
|Full                       |96017  |1.769     |
|Small                      |72571  |1.337     |
|Group                      |27025  |0.498     |
|Audit Exemption Subsidiary |25788  |0.475     |
|Total Exemption Small      |12266  |0.226     |
|Medium                     |1451   |0.027     |
|Audited abridged           |1437   |0.026     |
|No accounts type available |928    |0.017     |
|Filing Exemption Subsidiary|367    |0.007     |
|Interim                    |162    |0.003     |
|Partial Exemption          |29     |0.001     |
|Initial                    |4      |0.0       |
+-------------------

In [14]:
# Which SIC codes have the highest current assets on average?
companies.join(sic_codes, 'company_number', how='left')\
    .select([
        companies.company_number, 
        sic_codes.sic_code, 
        sic_codes.sic_description,
        companies.current_assets
    ])\
    .groupBy('sic_description')\
    .avg('current_assets')\
    .sort('avg(current_assets)', ascending=False)\
    .show(truncate=False)

+--------------------------------------------------------------------------------------------------+--------------------+
|sic_description                                                                                   |avg(current_assets) |
+--------------------------------------------------------------------------------------------------+--------------------+
|Banks                                                                                             |1.6769202507543105E8|
|Financial leasing                                                                                 |1.1421620102743903E8|
|Security dealing on own account                                                                   |3.2738479891555935E7|
|Activities of mortgage finance companies                                                          |2.472619615787771E7 |
|Treatment and disposal of hazardous waste                                                         |1.643513333625731E7 |
|Factoring              

In [15]:
companies.groupBy('company_type')\
    .avg('average_number_employees_during_period')\
    .withColumn('avg_employees', col('avg(average_number_employees_during_period)').cast('int'))\
    .drop('avg(average_number_employees_during_period)')\
    .show(truncate=False)

+--------------------------------------------------------------------------------------+-------------+
|company_type                                                                          |avg_employees|
+--------------------------------------------------------------------------------------+-------------+
|Converted / closed                                                                    |2            |
|Private unlimited company                                                             |11           |
|Public limited company                                                                |49           |
|Private limited company                                                               |206          |
|Private limited by guarantee without share capital                                    |251          |
|Private Limited Company by guarantee without share capital, use of 'Limited' exemption|21           |
|Private unlimited company without share capital                         

In [16]:
owners.filter(owners.is_owner == True)\
    .groupby(['company_country'])\
    .count()\
    .sort('count', ascending=False)\
    .show()


+--------------------+-------+
|     company_country|  count|
+--------------------+-------+
|                NULL|3908976|
|      united kingdom| 173234|
|            scotland|   3912|
|              jersey|   1891|
|       united states|   1799|
|virgin islands br...|   1548|
|            guernsey|   1452|
|         isle of man|    774|
|           hong kong|    717|
|               wales|    715|
|         netherlands|    694|
|    northern ireland|    694|
|          seychelles|    633|
|             ireland|    584|
|               china|    557|
|           gibraltar|    413|
|             germany|    391|
|    marshall islands|    369|
|              france|    359|
|              norway|    345|
+--------------------+-------+
only showing top 20 rows



In [17]:
owners.join(companies, owners.company_number == companies.company_number, how='left')\
    .groupby([companies.company_type, owners.officer_role])\
    .count()\
    .sort('count', ascending=False)\
    .show(truncate=False)

+--------------------------------------------------------------------------------------+---------------------+-------+
|company_type                                                                          |officer_role         |count  |
+--------------------------------------------------------------------------------------+---------------------+-------+
|Private limited company                                                               |Director             |7305028|
|Private limited company                                                               |NULL                 |1283809|
|Private limited company                                                               |Secretary            |1014104|
|Private limited by guarantee without share capital                                    |Director             |376686 |
|Private Limited Company by guarantee without share capital, use of 'Limited' exemption|Director             |138571 |
|Limited liability partnership                  

In [18]:
owners.groupBy('company_number')\
    .count()\
    .join(companies, 'company_number', how='left')\
    .select([
        'company_name',
        'count'
    ])\
    .sort('count', ascending=False)\
    .show(truncate=False)

+--------------------------------------------------------------------+-----+
|company_name                                                        |count|
+--------------------------------------------------------------------+-----+
|Albert Bowling Club Limited                                         |5    |
|Stadco Limited                                                      |5    |
|Newcastle Upon Tyne Law Society                                     |5    |
|Working Men's College Corporation                                   |5    |
|Sheffield Masonic Hall Company Limited.(the)                        |5    |
|Rye Cattle Market Company Limited(the)                              |5    |
|Public Picture Gallery Fund Birmingham                              |5    |
|Shorthorn Society Of The United Kingdom Of Great Britain And Ireland|5    |
|Peter Maurice Music Company Limited(the)                            |5    |
|Dvhl1 Limited                                                       |5    |

In [19]:
filings.groupBy([
    'category',
    'description'
    ])\
    .count()\
    .sort('count', ascending=False)\
    .show(truncate=False)

+-------------+---------------------------------------------------+-------+
|category     |description                                        |count  |
+-------------+---------------------------------------------------+-------+
|incorporation|Incorporation                                      |1484882|
|gazette      |First Gazette notice for compulsory strike-off     |402049 |
|gazette      |Compulsory strike-off action has been discontinued |326098 |
|accounts     |Micro company accounts made up to 2023-03-31       |280815 |
|accounts     |Micro company accounts made up to 2022-03-31       |259387 |
|accounts     |Total exemption full accounts made up to 2023-03-31|255485 |
|accounts     |Total exemption full accounts made up to 2022-03-31|237444 |
|dissolution  |Compulsory strike-off action has been suspended    |173529 |
|accounts     |Total exemption full accounts made up to 2022-12-31|151673 |
|accounts     |Micro company accounts made up to 2022-12-31       |145807 |
|accounts   

In [20]:
filings.groupBy('company_number')\
    .sum('pages')\
    .join(companies, 'company_number', how='left')\
    .select([
        'company_name',
        'company_number',
        'sum(pages)'
    ])\
    .sort('sum(pages)', ascending=False)\
    .show(truncate=False)

+------------------------------------------------------------+--------------+----------+
|company_name                                                |company_number|sum(pages)|
+------------------------------------------------------------+--------------+----------+
|Mediobanca - Banca Di Credito Finanziario Societa Per Azioni|FC028127      |4737      |
|Intesa Sanpaolo S.p.a                                       |FC016201      |4406      |
|Obrascon Huarte Lain S.a.                                   |FC034461      |3105      |
|NULL                                                        |AC001439      |2104      |
|Bayerische Landesbank                                       |FC009267      |2030      |
|International Consolidated Airlines Group S.a.              |FC029884      |1950      |
|Banco Comercial Portugues S.a.                              |FC026231      |1707      |
|Banco Santander S.a.                                        |FC004459      |1679      |
|Spw Investments Limi

In [21]:
incorporation_files = filings.join(companies, 'company_number', how='left')\
    .select(['company_name', 'date', 'incorporation_date'])\
    .withColumn('flag_incorporation_file', (col('date') == col('incorporation_date')).cast('int'))\
    .groupBy()\
    .sum('flag_incorporation_file').collect()[0][0]
incorporation_files / filings.count()

0.07061160440373027

In [22]:
spark.stop()